In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import os
import time
import random
import time
import requests 
import re
import logging
import subprocess
from subprocess import Popen
from sys import platform
import os, sys
import logging
import json
import threading

from netunicorn.client.remote import RemoteClient, RemoteClientException
from netunicorn.base import Experiment, ExperimentStatus, Pipeline
from netunicorn.library.tasks.basic import SleepTask
from netunicorn.library.tasks.measurements.ping import Ping
from netunicorn.base.architecture import Architecture
from netunicorn.base.nodes import Node
from netunicorn.base.task import Failure, Task, TaskDispatcher
from netunicorn.base import Result, Failure, Success, Task, TaskDispatcher
from netunicorn.base.architecture import Architecture
from netunicorn.base.nodes import Node

from typing import Dict
from typing import Optional
from enum import IntEnum
from datetime import datetime

# Discussion Section Week 6
In this section we will practice running experiments with netUnicorn.

First we will get you setup with your netUnicorn API Credentials. If you submitted a project proposal, you should have received an email containing the credentials for accessing the netunicorn API. Please log on to the server and query the length and amount of available nodes in this deployment.

## NetUnicorn API Credentials

In [3]:
NETUNICORN_ENDPOINT = os.environ.get('NETUNICORN_ENDPOINT', 'https://pinot.cs.ucsb.edu/netunicorn')
NETUNICORN_LOGIN = os.environ.get('NETUNICORN_LOGIN', '')       # substitute your login here
NETUNICORN_PASSWORD = os.environ.get('NETUNICORN_PASSWORD', '') # substitue your password here

Display the available nodes in the netunicorn deployment. You should nodes of a few different types (aws, raspi, etc.)

In [ ]:
client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD)
print("Health Check: {}".format(client.healthcheck()))
nodes = client.get_nodes()
print(nodes)
print(len(nodes))

Last week we talked about the SpeedTest Task and how we can run speedtests on nodes and retrieve the results. Similarly, we have the Ping Task that we can use to ping an address. We can also specify the number of ping samples to generate.

In [4]:
pipeline = Pipeline().then(Ping(address='8.8.8.8', count=5))

In [ ]:
client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD)
nodes = client.get_nodes()
working_nodes = nodes.filter(lambda node: node.name.startswith("raspi")).take(1)
experiment = Experiment().map(pipeline, working_nodes)
experiment_label = "test_ping"

try:
    client.delete_experiment(experiment_label)
except RemoteClientException:
    pass

# Prepare Experiment
client.prepare_experiment(experiment, experiment_label)
while True:
    info = client.get_experiment_status(experiment_label)
    print(info.status)
    if info.status == ExperimentStatus.READY:
        break
    time.sleep(20)

time.sleep(5)

# Execute Experiment
client.start_execution(experiment_label)
while True:
    info = client.get_experiment_status(experiment_label)
    print(info.status)
    if info.status != ExperimentStatus.RUNNING:
        break
    time.sleep(20)

In [ ]:
# Get Results
from returns.pipeline import is_successful
from returns.result import Failure

for report in info.execution_result:
    print(f"Node name: {report.node.name}")
    print(f"Error: {report.error}")

    if report.result is None:
        print("report.result is EMPTY..")
        continue

    result, log = report.result  # report stores results of execution and corresponding log

    # result is a returns.result.Result object, could be Success of Failure
    print(f"Result is: {type(result)}")
    if is_successful(result):
        data = result.unwrap()
    else:
        data = result.failure()
    try:
        for key, value in data.items():
            print(f"{key}: {value}")
    except:
        print(f"No attribute 'items' in result")

    # we also can explore logs
    for line in log:
        print(line.strip())
    print()

You can observe that the raw_output, as well as the parsed output in this case, is returned in the results

## Traceroute Task
For this task you will be implementing a traceroute task, similar to the [ping](https://github.com/netunicorn/netunicorn-library/blob/main/tasks/measurements/ping.py) task.

In [ ]:
import subprocess
from dataclasses import dataclass
from typing import List

from netunicorn.base.architecture import Architecture
from netunicorn.base.nodes import Node
from netunicorn.base.task import Failure, Task, TaskDispatcher

class TraceRoute(TaskDispatcher):
    def __init__(self, address: str, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.address = address
        self.linux_implementation = TraceRouteLinuxImplementation(self.address)

    def dispatch(self, node: Node) -> Task:
        if node.architecture in {Architecture.LINUX_AMD64, Architecture.LINUX_ARM64}:
            return self.linux_implementation
        raise NotImplementedError(
            f'TraceRoute is not implemented for architecture: {node.architecture}'
        )

class TraceRouteLinuxImplementation(Task):
    
    # Set the requirements to install the traceroute library
    requirements = [""]

    def __init__(self, address: str, *args, **kwargs):
        self.address = address.strip()
        super().__init__(*args, **kwargs)

    def run(self):
        # IMPLEMENT THIS FUNCTION
        return None
    
    def _format(self, output):
        # Optional: Parse the output
        return output

In [19]:
# Define the pipeline to just run this new task
pipeline = None

In [ ]:
client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD)
nodes = client.get_nodes()
working_nodes = nodes.filter(lambda node: node.name.startswith("raspi")).take(1)
experiment = Experiment().map(pipeline, working_nodes)
experiment_label = "test_traceroute"

try:
    client.delete_experiment(experiment_label)
except RemoteClientException:
    pass

# Prepare Experiment
client.prepare_experiment(experiment, experiment_label)
while True:
    info = client.get_experiment_status(experiment_label)
    print(info.status)
    if info.status == ExperimentStatus.READY:
        break
    time.sleep(20)

time.sleep(5)

# Execute Experiment
client.start_execution(experiment_label)
while True:
    info = client.get_experiment_status(experiment_label)
    print(info.status)
    if info.status != ExperimentStatus.RUNNING:
        break
    time.sleep(20)

In [ ]:
# Get Results
from returns.pipeline import is_successful
from returns.result import Failure

for report in info.execution_result:
    print(f"Node name: {report.node.name}")
    print(f"Error: {report.error}")

    if report.result is None:
        print("report.result is EMPTY..")
        continue

    result, log = report.result  # report stores results of execution and corresponding log

    # result is a returns.result.Result object, could be Success of Failure
    print(f"Result is: {type(result)}")
    if is_successful(result):
        data = result.unwrap()
    else:
        data = result.failure()
    try:
        for key, value in data.items():
            print(f"{key}: {value}")
    except:
        print(f"No attribute 'items' in result")

    # we also can explore logs
    for line in log:
        print(line.strip())
    print()

## File Download Task
For this task you will be implementing a task to download a single file using `wget` and reporting the time it takes to finish downloading.

In [ ]:
import subprocess
from dataclasses import dataclass
from typing import List

from netunicorn.base.architecture import Architecture
from netunicorn.base.nodes import Node
from netunicorn.base.task import Failure, Task, TaskDispatcher


class FileDownload(TaskDispatcher):
    def __init__(self, address: str, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.address = address
        self.linux_implementation = FileDownloadLinuxImplementation(self.address)

    def dispatch(self, node: Node) -> Task:
        if node.architecture in {Architecture.LINUX_AMD64, Architecture.LINUX_ARM64}:
            return self.linux_implementation
        raise NotImplementedError(
            f'TraceRoute is not implemented for architecture: {node.architecture}'
        )


class FileDownloadLinuxImplementation(Task):
    # Set the requirements to install the wget library
    requirements = [""]

    def __init__(self, address: str, *args, **kwargs):
        self.address = address.strip()
        super().__init__(*args, **kwargs)

    def run(self):
        # IMPLEMENT THIS FUNCTION

In [36]:
# Define the pipeline to just run this new task
pipeline = None

In [ ]:
client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD)
nodes = client.get_nodes()
working_nodes = nodes.filter(lambda node: node.name.startswith("raspi")).take(1)
experiment = Experiment().map(pipeline, working_nodes)
experiment_label = "test_traceroute"

try:
    client.delete_experiment(experiment_label)
except RemoteClientException:
    pass

# Prepare Experiment
client.prepare_experiment(experiment, experiment_label)
while True:
    info = client.get_experiment_status(experiment_label)
    print(info.status)
    if info.status == ExperimentStatus.READY:
        break
    time.sleep(20)

time.sleep(5)

# Execute Experiment
client.start_execution(experiment_label)
while True:
    info = client.get_experiment_status(experiment_label)
    print(info.status)
    if info.status != ExperimentStatus.RUNNING:
        break
    time.sleep(20)

In [ ]:
# Get Results
from returns.pipeline import is_successful
from returns.result import Failure

for report in info.execution_result:
    print(f"Node name: {report.node.name}")
    print(f"Error: {report.error}")

    if report.result is None:
        print("report.result is EMPTY..")
        continue

    result, log = report.result  # report stores results of execution and corresponding log

    # result is a returns.result.Result object, could be Success of Failure
    print(f"Result is: {type(result)}")
    if is_successful(result):
        data = result.unwrap()
    else:
        data = result.failure()
    try:
        for key, value in data.items():
            print(f"{key}: {value}")
    except:
        print(f"No attribute 'items' in result")

    # we also can explore logs
    for line in log:
        print(line.strip())
    print()

## SpeedTest + File Download
Now we will attempt to run a speedtest alongside this file download to demonstrate how these active measurements can affect user traffic. Create a pipeline that starts a speedtest task using the StartSpeedTest task defined below, and then attempts to download a file. How different are the observed results?

In [ ]:
import subprocess
from typing import Dict

from netunicorn.base.architecture import Architecture
from netunicorn.base.nodes import Node
from netunicorn.base.task import Failure, Task, TaskDispatcher


class StartSpeedTest(TaskDispatcher):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.linux_instance = StartSpeedTestLinuxImplementation(name=self.name)

    def dispatch(self, node: Node) -> Task:
        if node.architecture in {Architecture.LINUX_AMD64, Architecture.LINUX_ARM64}:
            return self.linux_instance

        raise NotImplementedError(
            f'SpeedTest is not implemented for architecture: {node.architecture}'
        )


class StartSpeedTestLinuxImplementation(Task):
    requirements = ["pip install speedtest-cli"]

    def run(self):
        proc = subprocess.Popen(["speedtest-cli", "--simple", "--secure"], capture_output=False)
        time.sleep(2)
        if (exit_code := proc.poll()) is None:  # not finished yet
            return Success(proc.pid)
        return Failure(f"Speedtest terminated with return code {exit_code}")


In [ ]:
# Define the pipeline to just run this experiment
pipeline = None

In [ ]:
client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD)
nodes = client.get_nodes()
working_nodes = nodes.filter(lambda node: node.name.startswith("raspi")).take(1)
experiment = Experiment().map(pipeline, working_nodes)
experiment_label = "test_traceroute"

try:
    client.delete_experiment(experiment_label)
except RemoteClientException:
    pass

# Prepare Experiment
client.prepare_experiment(experiment, experiment_label)
while True:
    info = client.get_experiment_status(experiment_label)
    print(info.status)
    if info.status == ExperimentStatus.READY:
        break
    time.sleep(20)

time.sleep(5)

# Execute Experiment
client.start_execution(experiment_label)
while True:
    info = client.get_experiment_status(experiment_label)
    print(info.status)
    if info.status != ExperimentStatus.RUNNING:
        break
    time.sleep(20)

In [ ]:
# Get Results
from returns.pipeline import is_successful
from returns.result import Failure

for report in info.execution_result:
    print(f"Node name: {report.node.name}")
    print(f"Error: {report.error}")

    if report.result is None:
        print("report.result is EMPTY..")
        continue

    result, log = report.result  # report stores results of execution and corresponding log

    # result is a returns.result.Result object, could be Success of Failure
    print(f"Result is: {type(result)}")
    if is_successful(result):
        data = result.unwrap()
    else:
        data = result.failure()
    try:
        for key, value in data.items():
            print(f"{key}: {value}")
    except:
        print(f"No attribute 'items' in result")

    # we also can explore logs
    for line in log:
        print(line.strip())
    print()

## SpeedTest + File Download
How do we need to change the implementation if we want to run a speedtest in the middle of a file download?